## Item 19: Never Unpack More Than Three Variables When Functions Return Multiple Values ##

Pretty simple, the point here is that using a large number of return values is extremely error prone, especially when you accidentally put unpacked variables in the wrong order.

## Item 20: Prefer Raising Exceptions to Returning `None`##

The point here is that you lose information by returning `None` instead of raising exceptions.  The motivating example is in the case of a helper method to handle zero division.  If you return `None` it's ambiguous as to whether something like a `str` was entered into the function or if you tried to divide by 0.  

In [7]:
def careful_divide(a, b):
    try:
        return a/b
    except ZeroDivisionError:
        return None
        
        
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print('Invalid Inputs')
    
# but what happens if the numerator is 0 and we evalute with an if statement?
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print('Invalid Inputs')
# we see it prints 'Invalid Inputs', but it shouldn't.    

Invalid Inputs
Invalid Inputs


**One way to make this better is to split the return value into a two-tuple, where the first part indicates that the operation was a success or failure, and the second part is the actual result.**

In [11]:
def careful_divide(a, b):
    try:
        return True, a/b
    except ZeroDivisionError:
        return False, None
        
        
x, y = 1, 0
success, result = careful_divide(x, y)
if not success:
    print(f'Invalid Inputs of {x,y}')
    
# but what happens if the numerator is 0 and we evalute with an if statement?
x, y = 0, 5
success, result = careful_divide(x, y)
if not success:
    print(f'Invalid Inputs {x,y}')

Invalid Inputs of (1, 0)


**The second, better way to reduce errors is to never return `None` for special cases.  Instead, raise an `Exception` up to the caller and have the caller deal with it.**

In [13]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid Inputs')
        
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid Inputs')
else:
    print(f'Result is {result}')

Result is 2.5


**We can further extend this approach using type annotations to help clear up what type the inputs and outputs should be (but note that these are ignored by the interpreter and only serve as notes to users down the line):**

In [19]:
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

try:
    result = careful_divide(1, 0)
    #assert False
except ValueError:
    print('Caught the value error')

assert careful_divide(1, 5) == 0.2

Caught the value error


## Item 21: Know How Closures Interact With Variable Scope ##

### Motivating Example: ### 
Say that I want to sort a `list` of numbers but prioritize one group of numbers to come first.  A common way to do this is to pass a helper function as the key argument to a list's `sort` method.

In [20]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return(0, x)
        return(1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


The above example works for three reasons:
1. Python supports **closures** - that is, functions that refer to variables from the scope in which they were defined.  This is why the helper function is able to access the group argument for `sort_priority`.  
2. Functions are **first-class** objects in Python, which means you can refer to them directly, assign them to variables, pass them as arguments to other functions, compare them in expressions and if statements, and os on.  This is how the `sort` method can accept a closure function as the key argument.  
3. Python has specific rules for comparing sequences.  It first compares items at index zero; then, if those are equal, it compares items at index one; if they are still equal, it compares items at index two, and so on.  This is why the return value from the `helper` closure causes the sort order to have two distinct groups.

### Let's try to extend our function to return whether higher-priority items were seen at all so the user interface code can act accordingly and see how scope resolution works and how to use the `nonlocal` statement to indicate when a closure can modify a variable in its enclosing scope.### 

In [21]:
def sort_priority2(numbers, group):
    found = False   # scope: set_priority2
    def helper(x):
        if x in group:
            found = True   # scope: helper
            return(0, x)
        return(1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print(f'Found: {found}')
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


**The issue is that because of scope resolution, the closure cannot modify the found variable in its enclosing scope, only in its local scope, even though it has access to it.  In Python you can use the `nonlocal` statement is used to indicate that scope traversal should happen upon assignment for a specific variable name.**  
  
As with the anti-pattern of global variables, use `nonlocal` very sparingly and only for very simple functions.  If things get any more complicated than the most simple of cases, use a `class` instead.


In [23]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found # found added
        if x in group:
            found = True
            return(0, x)
        return(1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority3(numbers, group)
print(f'Found: {found}')
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


## Item 22: Reduce Visual Noise with Variable Positional Arguments ##

Use a variable number of positional arguments, `*args`, pronounced "star args" to help make things clearer and reduce visual noise.

In [24]:
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')
        
log('My numbers are', 1, 2)
log('Hi there')

My numbers are: 1, 2
Hi there


There are two problems that arise with accepting a variable number of positional arguments:
    
    1. The optional positional arguments are always turned into a `tuple` before they are passed to a function
    2. You can't add new positional arguments to a function in the future without migrating every caller. 
        a. If you try to add a positional argument in the front of the argument list, existing callers will subtly break if they aren't updated with the correct input arguments.

## Item 23: Provide Optional Behavior with Keyword Arguments ##

Python functions accept passing arguments by position as well as by keyword.  

In [29]:
def remainder(number, divisor):
    return number % divisor

assert remainder(20, 7) == 6

# All of these are equivalent because of keyword arguments.
print(remainder(20, 7))
print(remainder(20, divisor=7))
print(remainder(number=20, divisor=7))
print(remainder(divisor=7, number=20))

6
6
6
6


In [30]:
# But note that positional arguments must be specified before keyword arguments:
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (<ipython-input-30-60e2f4c14efe>, line 2)

**If you already have a dictionary, and you want to use its contents to call a function like `remainder`, you can do this by using the ** operator.  This instructs Python to pass the values from the dictionary as the corresponding keyword arguments of the function:**

In [34]:
# keyword argument dictionary
my_kwargs = {
    'number': 20,
    'divisor': 7
}

print(f'remainder(**my_kwargs) = {remainder(**my_kwargs)}')

# you can also mix the ** operator with positional arguments or keyword arguments in the function call
# as long as no argument is repeated:

my_kwargs = {
    'divisor': 7
}

print(f'remainder(number=20, **my_kwargs) = {remainder(number=20, **my_kwargs)}')

remainder(**my_kwargs) = 6
remainder(number=20, **my_kwargs) = 6


There are three main benefits to using keyword arguments:
    1. They make the function call clearer to new readers of the code
    2. They can have default values specified in the function definition
    3. They provide a powerful way to extend a function's parameters while remaining backward compatible with existing callers.

## Item 24: Use `None` and Docstrings to Specify Dyanmic Default Arguments##

### Motivating Example: ###
Let's say I want to use a non-static type as a keyword argument's default value.  A naive (non-working) way to try to achieve this would be:


In [36]:
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')
    
log('Hi there!')
sleep(0.1)
log('Hello again!')

2021-08-07 13:28:37.575121: Hi there!
2021-08-07 13:28:37.575121: Hello again!


#### The problem is that a default argument value is evaluated only once per module load, which usually happens when a program starts up.  After the module containing this code is loaded, the `datetime.now()` default argument will never be evaluated again.  

#### The convention for achieving the desired result in Python is to provide a default value of `None` and to document the actual behavior in the docstring.  When your code sees the argument values `None`, you allocate the default value accordingly: 

In [37]:
def log(message, when=None):
    """
    Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log('Hi there!')
sleep(0.1)
log('Hello again!')

2021-08-07 13:32:08.982597: Hi there!
2021-08-07 13:32:09.083202: Hello again!


### We can also extend this approach using type annotations to make things a bit clearer to future users:

In [41]:
from typing import Optional

def log_typed(message: str, 
              when: Optional[datetime]=None) -> None:
    """
    Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log_typed('Hi there!')
sleep(0.1)
log_typed('Hello again!')

2021-08-07 13:38:15.128135: Hi there!
2021-08-07 13:38:15.228864: Hello again!
